In [3]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import numpy as np

from brokenaxes import brokenaxes

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
# CSV 파일에 접근
maml_info = pd.read_csv('../MAML_5way_1shot/MAML_5way_1shot_inner_loop_test.csv')
arbiter_info = pd.read_csv('../MAML+Arbiter_5way_1shot/MAML+Arbiter_5way_1shot_inner_loop_test.csv')
# alfa_info = pd.read_csv('../MAML+ALFA/alfa+maml_inner_loop_test.csv')
# l2f_info = pd.read_csv('../MAML+L2F/MAML+L2F_inner_loop_test.csv')

# 훈련 정보만 가지고 온다
maml_info_train = maml_info[maml_info['phase']=='val']
arbiter_info_train= arbiter_info[arbiter_info['phase']=='val']
# alfa_info = alfa_info[alfa_info['phase'] == 'val']
# l2f_info = l2f_info[alfa_info['phase'] == 'val']

FileNotFoundError: [Errno 2] No such file or directory: '../MAML+Arbiter_5way_1shot/MAML+Arbiter_5way_1shot_inner_loop_test.csv'

# 1. 정보를 Domain 별로 압축

In [ ]:
maml_info_mini_imagenet = maml_info_train[maml_info_train['current_iter']=='mini_imagenet_full_size']
maml_info_mini_imagenet = maml_info_mini_imagenet.groupby('num_step')


# maml_info_cifar_fs = maml_info_train[maml_info_train['current_iter']=='CIFAR_FS']
# maml_info_cifar_fs = maml_info_cifar_fs.groupby('num_step')

# maml_info_CUB = maml_info_train[maml_info_train['current_iter']=='CUB']
# maml_info_CUB = maml_info_CUB.groupby('num_step')

In [ ]:
# mini-imagenet
arbiter_info_mini_imagenet = arbiter_info[arbiter_info['current_iter']=='mini_imagenet_full_size']
arbiter_info_mini_imagenet = arbiter_info_mini_imagenet.groupby('num_step')

# # tiered-imagenet
# arbiter_info_tiered_imagenet = arbiter_info[arbiter_info['current_iter']=='tiered_imagenet']
# arbiter_info_tiered_imagenet = arbiter_info_tiered_imagenet.groupby('num_step')
# # CIFAR_FS
# arbiter_info_CIFAR_FS = arbiter_info[arbiter_info['current_iter']=='CIFAR_FS']
# arbiter_info_CIFAR_FS = arbiter_info_CIFAR_FS.groupby('num_step')
# # CUB
# arbiter_info_CUB = arbiter_info[arbiter_info['current_iter']=='CUB']
# arbiter_info_CUB = arbiter_info_CUB.groupby('num_step')

# 'layer_dict.conv0.conv.weight_grad_L2norm', 
# 'layer_dict.conv1.conv.weight_grad_L2norm', 
# 'layer_dict.conv2.conv.weight_grad_L2norm',
# 'layer_dict.conv3.conv.weight_grad_L2norm',
# 'layer_dict.linear.weights_grad_L2norm'

In [ ]:
group_names = []

for name, group in maml_info_mini_imagenet:
    # 그룹 이름을 저장 (모든 그룹은 동일한 순서로 추가됨)
    if name not in group_names:
        group_names.append(str(int(name)+1))

x = range(len(group_names))
        
maml_mean_df = maml_info_mini_imagenet['loss_mean'].mean()
arbiter_mean_df = arbiter_info_mini_imagenet['loss_mean'].mean()

maml_std_devs = maml_info_mini_imagenet['loss_std'].mean()
arbiter_std_devs= arbiter_info_mini_imagenet['loss_std'].mean()

plt.figure(figsize=(10, 6))

plt.plot(x, maml_mean_df, '-o', color='b')#, label='MAML')
plt.fill_between(x, maml_mean_df - maml_std_devs, maml_mean_df + maml_std_devs, color='b', alpha=0.1, label='MAML')
plt.plot(x, arbiter_mean_df, '-o', color='r')#, label='MetaGNA')
plt.fill_between(x, arbiter_mean_df - arbiter_std_devs, arbiter_mean_df + arbiter_std_devs, color='r', alpha=0.1, label='MetaGNA')

# plt.title('Loss Distribution')
plt.xticks(x, group_names)
plt.legend(fontsize=15)
plt.xlabel('Inner-loop step',fontsize=15)
plt.ylabel('value', fontsize=14)
plt.xticks(fontsize=15)  # x축 값 크기
plt.yticks(fontsize=15)  # y축 값 크기

plt.savefig('loss_distribution' + '.png')

In [ ]:
group_names = []

for name, group in maml_info_mini_imagenet:
    # 그룹 이름을 저장 (모든 그룹은 동일한 순서로 추가됨)
    if name not in group_names:
        group_names.append(str(int(name)+1))

x = range(len(group_names))
        
maml_mean_df = maml_info_mini_imagenet['all_grads_l2norm'].mean()
# before_arbiter_mean_df = arbiter_info_mini_imagenet['pre_all_grads_l2norm'].mean()
arbiter_mean_df = arbiter_info_mini_imagenet['all_grads_l2norm'].mean()

maml_means = maml_mean_df.values
arbiter_means = arbiter_mean_df.values

# 데이터에 로그 변환 적용
maml_means = np.log(maml_means)
arbiter_means = np.log(arbiter_means)
# arbiter_withoutWN_means = np.log(arbiter_withoutWN_means)

# 꺾은선 그래프를 그립니다
plt.figure(figsize=(10, 6))
plt.plot(x, maml_means, '-o', color='b', label='MAML')
# plt.plot(x, before_arbiter_mean_df, '-o', color='g', label='before_Arbiter')
plt.plot(x, arbiter_means, '-o', color='r', label='Arbiter')

plt.legend()


# plt.title('Gradient Norm')
plt.xticks(x, group_names)
plt.legend(fontsize=15)
plt.xlabel('Inner-loop step',fontsize=15)
plt.ylabel('value', fontsize=14)
plt.xticks(fontsize=15)  # x축 값 크기
plt.yticks(fontsize=15)  # y축 값 크기

plt.savefig('gradient_norm' + '.png')

In [ ]:
group_names = []

for name, group in maml_info_mini_imagenet:
    # 그룹 이름을 저장 (모든 그룹은 동일한 순서로 추가됨)
    if name not in group_names:
        group_names.append(str(int(name)+1))

x = range(len(group_names))
        
maml_mean_df = maml_info_mini_imagenet['gsnr'].mean()
#arbiter_mean_df = arbiter_info_mini_imagenet['pre_gsnr'].mean()
arbiter_mean_df = arbiter_info_mini_imagenet['gsnr'].mean()
# l2f_mean_df = l2f_info['gsnr'].mean()
# alfa_mean_df = alfa_info['gsnr'].mean()

maml_means = maml_mean_df.values
arbiter_means = arbiter_mean_df.values
#l2f_mean_df = l2f_mean_df.values

# 데이터에 로그 변환 적용
# maml_means = np.log(maml_means)
# arbiter_means = np.log(arbiter_means)
# arbiter_withoutWN_means = np.log(arbiter_withoutWN_means)

# 꺾은선 그래프를 그립니다
plt.figure(figsize=(10, 6))
plt.plot(x, maml_means, '-o', color='b', label='MAML')
plt.plot(x, arbiter_means, '-o', color='r', label='Arbiter')
#plt.plot(x, l2f_mean_df, '-o', color='g', label='L2F')
# plt.plot(x, arbiter_withoutWN_means, '-o', color='g', label='Arbiter_withoutWN')
# plt.plot(x, arbiter_withoutGN_means, '-o', color='violet', label='Arbiter_withoutGN')

# plt.title('GSNR')
plt.xticks(x, group_names)
plt.legend(fontsize=15)
plt.xlabel('Inner-loop step',fontsize=15)
plt.ylabel('value', fontsize=14)
plt.xticks(fontsize=15)  # x축 값 크기
plt.yticks(fontsize=15)  # y축 값 크기

plt.savefig('gsnr' + '.png')